# [intro ](https://www.tensorflow.org/guide/keras/serialization_and_saving)
一个模型有以下组成部分：  
1. architecture（layers and how they connected） 
2. weights values
3. optimizer (defined by compiling the model)
4. losses and metrics (defined by compiling the model or calling add_loss() or add_metric())

# whole save & load

一般情况下都会选择保存整个模型，除非是在训练过程中保存checkpoints。整体保存&加载的API如下：

- model.save() or tf.keras.models.save_model()
- tf.keras.models.load_model()

可以保存成**the TensorFlow SavedModel format**，或者**H5 format**,推荐前者。后者也可以通过argument切换。

## SavedModel

保存后文件夹里又出现以下文件：
- assets  
- fingerprint.pb  
- keras_metadata.pb  
- saved_model.pb  (contains The model architecture, training configuration (including the optimizer, losses, and metrics))
- variables (contains weights)

对于自定义对象（eg.custom layers），会保存class name, call function, losses, weights (the config, 若有). 在没有config的情况下，call function用于创建一个像原始模型一样存在的模型，可以训练、评估和用于推理。但在编写自定义layer、model时最好也定义get_config和 from_config 。

# save weights
用途：
1. You only need the model for inference: in this case you won't need to restart training, so you don't need the compilation information or optimizer state.
2. You are doing transfer learning: in this case you will be training a new model reusing the state of a prior model, so you don't need the compilation information of the prior model.

一般保存为ckpt 
```python
sequential_model = keras.Sequential(
    [
        keras.Input(shape=(784,), name="digits"),
        keras.layers.Dense(64, activation="relu", name="dense_1"),
        keras.layers.Dense(64, activation="relu", name="dense_2"),
        keras.layers.Dense(10, name="predictions"),
    ]
)
sequential_model.save_weights("ckpt")
load_status = sequential_model.load_weights("ckpt")

# `assert_consumed` can be used as validation that all variable values have been
# restored from the checkpoint. 
load_status.assert_consumed()
```